# GCN architecture and hyperparameter tuning
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
from time import time

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.137562
std,1.740254
min,0.004530
25%,0.010635
50%,0.016778
75%,0.032392
max,46.419552


## 2. Filter

In [3]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.019962
std,0.013149
min,0.004530
25%,0.010309
50%,0.014975
75%,0.026453
max,0.067472


## 3. Grid search over architecture, learning rate, heads

In [5]:
saveDir = './results/modelTuning/architectureTuning03/'

# build training plan
architectureList = ['L16/C32/C64/C128/C256/C256/C128/L64/L2',
                    'L16/C32/C64/C128/C256/C512/C256/C128/L64/L2',
                    'L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2']
learningRatelist = [1e-4,1e-3,1e-2]
headList = [2,6]
seedList = [1,2,3]
trainConfigs = []
for ar in architectureList:
    for lr in learningRatelist:
        for he in headList:
            for s in seedList:
                trainConfigs.append({'Architecture': ar, 'Learning Rate': lr, '# of Heads': he, 'Trial': s})


# train models
for i,c in enumerate(trainConfigs):
    # partition data
    trainData, valData, testData = partitionGraphList(allGraphs, seed=c['Trial'])
    
    print(c)
    gcn = FeaStNet(architecture=c['Architecture'], heads=c['# of Heads'], useXFeatures=True, numXFeatures=3)
    c['Parameters'] = gcn.getNumParams()
    
    t = time()
    history = gcn.trainModel(trainData, valData, 
                             epochs=100, 
                             lr=c['Learning Rate'],
                             batchSize=256, 
                             flatten=True, 
                             logTrans=False, 
                             ssTrans=True, 
                             binaryLoads=True,
                             saveDir=saveDir+f'/{i:03}/')
    c['Train Time'] = time()-t
    plotHistory(history)
    
    # test
    trainRes = gcn.testModel(trainData)
    c['Train MSE'] = trainRes['mse']
    
    testRes = gcn.testModel(testData)
    c['Test MSE'] = testRes['mse']
    print(c['Test MSE'])

    # log
    df = pd.DataFrame(trainConfigs)
    df.to_csv(saveDir+'testResults.csv', index=False)

{'Architecture': 'L16/C32/C64/C128/C256/C256/C128/L64/L2', 'Learning Rate': 0.0001, '# of Heads': 2, 'Trial': 1}
epoch: 0   trainLoss: 9.8483e-01   valLoss:8.8174e-01  time: 1.09e+00
epoch: 1   trainLoss: 8.7466e-01   valLoss:8.8310e-01  time: 1.09e+00
epoch: 2   trainLoss: 8.2602e-01   valLoss:8.7749e-01  time: 1.05e+00
epoch: 3   trainLoss: 8.1900e-01   valLoss:8.5580e-01  time: 1.05e+00
epoch: 4   trainLoss: 8.1099e-01   valLoss:8.8411e-01  time: 1.03e+00
epoch: 5   trainLoss: 7.9722e-01   valLoss:8.5818e-01  time: 1.02e+00
epoch: 6   trainLoss: 7.4128e-01   valLoss:7.7835e-01  time: 1.03e+00
epoch: 7   trainLoss: 6.7536e-01   valLoss:7.1980e-01  time: 1.03e+00
epoch: 8   trainLoss: 6.9977e-01   valLoss:6.8714e-01  time: 1.02e+00
epoch: 9   trainLoss: 6.4665e-01   valLoss:6.6912e-01  time: 1.02e+00
epoch: 10   trainLoss: 6.2018e-01   valLoss:6.5772e-01  time: 1.04e+00
epoch: 11   trainLoss: 6.2425e-01   valLoss:6.4689e-01  time: 1.03e+00
epoch: 12   trainLoss: 6.1162e-01   valLoss:6

epoch: 12   trainLoss: 6.1536e-01   valLoss:5.6363e-01  time: 1.02e+00
epoch: 13   trainLoss: 6.0685e-01   valLoss:5.4869e-01  time: 1.02e+00
epoch: 14   trainLoss: 6.1122e-01   valLoss:5.3342e-01  time: 1.02e+00
epoch: 15   trainLoss: 5.8401e-01   valLoss:5.2077e-01  time: 1.03e+00
epoch: 16   trainLoss: 5.6700e-01   valLoss:5.0466e-01  time: 1.01e+00
epoch: 17   trainLoss: 5.2125e-01   valLoss:4.9022e-01  time: 1.03e+00
epoch: 18   trainLoss: 5.1341e-01   valLoss:4.7437e-01  time: 1.02e+00
epoch: 19   trainLoss: 5.2236e-01   valLoss:4.5759e-01  time: 1.03e+00
epoch: 20   trainLoss: 4.7397e-01   valLoss:4.4428e-01  time: 1.02e+00
epoch: 21   trainLoss: 4.5978e-01   valLoss:4.2738e-01  time: 1.04e+00
epoch: 22   trainLoss: 4.4655e-01   valLoss:4.1117e-01  time: 1.04e+00
epoch: 23   trainLoss: 4.2867e-01   valLoss:4.0017e-01  time: 1.02e+00
epoch: 24   trainLoss: 4.2745e-01   valLoss:3.8991e-01  time: 1.02e+00
epoch: 25   trainLoss: 4.1639e-01   valLoss:3.7712e-01  time: 1.02e+00
epoch:

epoch: 26   trainLoss: 4.1987e-01   valLoss:3.9957e-01  time: 9.97e-01
epoch: 27   trainLoss: 4.0907e-01   valLoss:3.9505e-01  time: 1.01e+00
epoch: 28   trainLoss: 4.0620e-01   valLoss:3.9230e-01  time: 9.91e-01
epoch: 29   trainLoss: 3.9465e-01   valLoss:3.8440e-01  time: 9.85e-01
epoch: 30   trainLoss: 3.8214e-01   valLoss:3.8239e-01  time: 9.79e-01
epoch: 31   trainLoss: 3.6400e-01   valLoss:3.7824e-01  time: 9.87e-01
epoch: 32   trainLoss: 3.7721e-01   valLoss:3.5915e-01  time: 1.02e+00
epoch: 33   trainLoss: 3.4264e-01   valLoss:3.4062e-01  time: 1.03e+00
epoch: 34   trainLoss: 3.4282e-01   valLoss:3.4071e-01  time: 1.02e+00
epoch: 35   trainLoss: 3.3343e-01   valLoss:3.4382e-01  time: 9.86e-01
epoch: 36   trainLoss: 3.2302e-01   valLoss:3.3756e-01  time: 9.95e-01
epoch: 37   trainLoss: 3.2494e-01   valLoss:3.2600e-01  time: 9.83e-01
epoch: 38   trainLoss: 3.0547e-01   valLoss:3.2242e-01  time: 9.77e-01
epoch: 39   trainLoss: 2.7346e-01   valLoss:3.2500e-01  time: 9.82e-01
epoch:

epoch: 40   trainLoss: 2.3242e-01   valLoss:3.3402e-01  time: 1.09e+00
epoch: 41   trainLoss: 2.0540e-01   valLoss:3.0143e-01  time: 1.11e+00
epoch: 42   trainLoss: 2.0729e-01   valLoss:2.8762e-01  time: 1.12e+00
epoch: 43   trainLoss: 1.9604e-01   valLoss:2.6492e-01  time: 1.12e+00
epoch: 44   trainLoss: 1.8991e-01   valLoss:2.7102e-01  time: 1.13e+00
epoch: 45   trainLoss: 2.0811e-01   valLoss:2.8587e-01  time: 1.11e+00
epoch: 46   trainLoss: 1.8858e-01   valLoss:2.6532e-01  time: 1.10e+00
epoch: 47   trainLoss: 1.7500e-01   valLoss:2.5470e-01  time: 1.12e+00
epoch: 48   trainLoss: 1.8054e-01   valLoss:2.6160e-01  time: 1.15e+00
epoch: 49   trainLoss: 1.7020e-01   valLoss:2.7374e-01  time: 1.13e+00
epoch: 50   trainLoss: 1.8304e-01   valLoss:2.5256e-01  time: 1.13e+00
epoch: 51   trainLoss: 1.5381e-01   valLoss:2.6868e-01  time: 1.13e+00
epoch: 52   trainLoss: 1.5842e-01   valLoss:2.4251e-01  time: 1.13e+00
epoch: 53   trainLoss: 1.6366e-01   valLoss:2.5640e-01  time: 1.13e+00
epoch:

epoch: 54   trainLoss: 1.4090e-01   valLoss:1.8177e-01  time: 1.12e+00
epoch: 55   trainLoss: 1.2444e-01   valLoss:1.7384e-01  time: 1.11e+00
epoch: 56   trainLoss: 1.2153e-01   valLoss:1.7657e-01  time: 1.11e+00
epoch: 57   trainLoss: 1.2536e-01   valLoss:1.8600e-01  time: 1.11e+00
epoch: 58   trainLoss: 1.1052e-01   valLoss:1.6453e-01  time: 1.13e+00
epoch: 59   trainLoss: 1.1105e-01   valLoss:1.7256e-01  time: 1.13e+00
epoch: 60   trainLoss: 1.2410e-01   valLoss:1.7503e-01  time: 1.12e+00
epoch: 61   trainLoss: 1.0349e-01   valLoss:1.9295e-01  time: 1.12e+00
epoch: 62   trainLoss: 1.0009e-01   valLoss:1.5540e-01  time: 1.13e+00
epoch: 63   trainLoss: 9.3618e-02   valLoss:1.9466e-01  time: 1.13e+00
epoch: 64   trainLoss: 1.0314e-01   valLoss:1.7428e-01  time: 1.12e+00
epoch: 65   trainLoss: 9.2624e-02   valLoss:1.5343e-01  time: 1.12e+00
epoch: 66   trainLoss: 1.0033e-01   valLoss:1.7576e-01  time: 1.13e+00
epoch: 67   trainLoss: 8.4496e-02   valLoss:1.4819e-01  time: 1.11e+00
epoch:

...

epoch: 90   trainLoss: 9.5266e-02   valLoss:2.2197e-01  time: 1.45e+00
epoch: 91   trainLoss: 1.0374e-01   valLoss:1.5279e-01  time: 1.40e+00
epoch: 92   trainLoss: 1.1283e-01   valLoss:1.6379e-01  time: 1.42e+00
epoch: 93   trainLoss: 1.0308e-01   valLoss:1.2917e-01  time: 1.43e+00
epoch: 94   trainLoss: 1.0476e-01   valLoss:1.7310e-01  time: 1.41e+00
epoch: 95   trainLoss: 8.4807e-02   valLoss:1.0303e-01  time: 1.38e+00
epoch: 96   trainLoss: 9.0388e-02   valLoss:1.0989e-01  time: 1.39e+00
epoch: 97   trainLoss: 1.0180e-01   valLoss:1.6333e-01  time: 1.42e+00
epoch: 98   trainLoss: 1.0751e-01   valLoss:2.2629e-01  time: 1.42e+00
epoch: 99   trainLoss: 1.0153e-01   valLoss:1.7925e-01  time: 1.37e+00
loading checkpoint 95
1.5955218e-05
{'Architecture': 'L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2', 'Learning Rate': 0.01, '# of Heads': 2, 'Trial': 2}
epoch: 0   trainLoss: 1.1163e+00   valLoss:8.4279e-01  time: 1.54e+00
epoch: 1   trainLoss: 8.4591e-01   valLoss:1.4088e+00  time: 1.

epoch: 2   trainLoss: 7.9396e-01   valLoss:1.1601e+00  time: 1.43e+00
epoch: 3   trainLoss: 6.6353e-01   valLoss:1.5680e+00  time: 1.43e+00
epoch: 4   trainLoss: 6.2327e-01   valLoss:1.0735e+00  time: 1.42e+00
epoch: 5   trainLoss: 5.5629e-01   valLoss:7.0270e-01  time: 1.44e+00
epoch: 6   trainLoss: 4.9917e-01   valLoss:9.9223e-01  time: 1.45e+00
epoch: 7   trainLoss: 4.4055e-01   valLoss:7.5188e-01  time: 1.44e+00
epoch: 8   trainLoss: 4.5519e-01   valLoss:6.4163e-01  time: 1.44e+00
epoch: 9   trainLoss: 4.1900e-01   valLoss:5.3745e-01  time: 1.43e+00
epoch: 10   trainLoss: 4.5853e-01   valLoss:4.9339e-01  time: 1.43e+00
epoch: 11   trainLoss: 3.6899e-01   valLoss:7.5930e-01  time: 1.41e+00
epoch: 12   trainLoss: 3.8880e-01   valLoss:1.1200e+00  time: 1.41e+00
epoch: 13   trainLoss: 4.7961e-01   valLoss:4.9743e-01  time: 1.42e+00
epoch: 14   trainLoss: 3.6498e-01   valLoss:5.0627e-01  time: 1.41e+00
epoch: 15   trainLoss: 3.3345e-01   valLoss:4.2649e-01  time: 1.41e+00
epoch: 16   tr

epoch: 16   trainLoss: 3.6548e-01   valLoss:5.9731e-01  time: 1.80e+00
epoch: 17   trainLoss: 3.7285e-01   valLoss:6.1859e-01  time: 1.79e+00
epoch: 18   trainLoss: 3.4710e-01   valLoss:7.4228e-01  time: 1.77e+00
epoch: 19   trainLoss: 3.6177e-01   valLoss:5.4118e-01  time: 1.80e+00
epoch: 20   trainLoss: 3.1526e-01   valLoss:7.0160e-01  time: 1.80e+00
epoch: 21   trainLoss: 3.5414e-01   valLoss:5.8495e-01  time: 1.78e+00
epoch: 22   trainLoss: 2.6547e-01   valLoss:4.3368e-01  time: 1.77e+00
epoch: 23   trainLoss: 2.6697e-01   valLoss:4.9222e-01  time: 1.78e+00
epoch: 24   trainLoss: 3.3587e-01   valLoss:9.6695e-01  time: 1.75e+00
epoch: 25   trainLoss: 3.0008e-01   valLoss:4.6561e-01  time: 1.78e+00
epoch: 26   trainLoss: 2.7438e-01   valLoss:5.1343e-01  time: 1.79e+00
epoch: 27   trainLoss: 2.5731e-01   valLoss:6.3345e-01  time: 1.80e+00
epoch: 28   trainLoss: 2.5334e-01   valLoss:5.3703e-01  time: 1.77e+00
epoch: 29   trainLoss: 2.8438e-01   valLoss:8.7289e-01  time: 1.79e+00
epoch:

epoch: 30   trainLoss: 2.8266e-01   valLoss:3.2534e-01  time: 1.77e+00
epoch: 31   trainLoss: 2.5856e-01   valLoss:5.4398e-01  time: 1.77e+00
epoch: 32   trainLoss: 2.4136e-01   valLoss:5.0669e-01  time: 1.78e+00
epoch: 33   trainLoss: 2.3764e-01   valLoss:3.8776e-01  time: 1.80e+00
epoch: 34   trainLoss: 2.1061e-01   valLoss:3.8990e-01  time: 1.77e+00
epoch: 35   trainLoss: 1.8569e-01   valLoss:4.4447e-01  time: 1.78e+00
epoch: 36   trainLoss: 2.0575e-01   valLoss:5.2278e-01  time: 1.80e+00
epoch: 37   trainLoss: 1.8769e-01   valLoss:5.0192e-01  time: 1.77e+00
epoch: 38   trainLoss: 2.2566e-01   valLoss:4.5078e-01  time: 1.80e+00
epoch: 39   trainLoss: 1.9783e-01   valLoss:3.7414e-01  time: 1.81e+00
epoch: 40   trainLoss: 1.8111e-01   valLoss:4.0717e-01  time: 1.79e+00
epoch: 41   trainLoss: 1.7822e-01   valLoss:2.9558e-01  time: 1.78e+00
epoch: 42   trainLoss: 1.9068e-01   valLoss:3.4073e-01  time: 1.77e+00
epoch: 43   trainLoss: 1.7477e-01   valLoss:5.4383e-01  time: 1.78e+00
epoch:

epoch: 44   trainLoss: 1.8642e-01   valLoss:3.2934e-01  time: 1.77e+00
epoch: 45   trainLoss: 1.8228e-01   valLoss:4.0770e-01  time: 1.80e+00
epoch: 46   trainLoss: 1.9990e-01   valLoss:2.4720e+00  time: 1.76e+00
epoch: 47   trainLoss: 2.6431e-01   valLoss:1.1454e+00  time: 1.80e+00
epoch: 48   trainLoss: 1.8010e-01   valLoss:2.8430e-01  time: 1.77e+00
epoch: 49   trainLoss: 1.6393e-01   valLoss:2.3372e-01  time: 1.75e+00
epoch: 50   trainLoss: 1.4707e-01   valLoss:4.0855e-01  time: 1.78e+00
epoch: 51   trainLoss: 1.6247e-01   valLoss:3.9997e-01  time: 1.80e+00
epoch: 52   trainLoss: 1.3434e-01   valLoss:4.9516e-01  time: 1.75e+00
epoch: 53   trainLoss: 1.2867e-01   valLoss:2.8997e-01  time: 1.76e+00
epoch: 54   trainLoss: 1.6776e-01   valLoss:2.7086e-01  time: 1.77e+00
epoch: 55   trainLoss: 1.5767e-01   valLoss:3.5472e-01  time: 1.79e+00
epoch: 56   trainLoss: 1.5722e-01   valLoss:3.7521e-01  time: 1.80e+00
epoch: 57   trainLoss: 1.4019e-01   valLoss:2.8863e-01  time: 1.81e+00
epoch:

In [6]:
# 27 configurations averaged over three trials
df.groupby(['Architecture', 'Learning Rate', '# of Heads']).mean()[['Parameters', 'Train Time','Train MSE', 'Test MSE']]

Parameters  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0001        2               296242   
                                                               6               866570   
                                                 0.0010        2               296242   
                                                               6               866570   
                                                 0.0100        2               296242   
                                                               6               866570   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        2               692020   
                                                               6              2050832   
                                                 0.0010        2               692020   
                                                               6              2050832   
                                                 0.0100        2               692020   
                                                               6              2050832   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0001        2              1218870   
                                                               6              3628310   
                                                 0.0010        2              1218870   
                                                               6              3628310   
                                                 0.0100        2              1218870   
                                                               6              3628310   

                                                                           Train Time  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0001        2           103.711840   
                                                               6           113.659523   
                                                 0.0010        2           101.490790   
                                                               6           111.179006   
                                                 0.0100        2           100.647021   
                                                               6           110.972384   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      0.0001        2           121.806531   
                                                               6           144.639770   
                                                 0.0010        2           120.791977   
                                                               6           143.831214   
                                                 0.0100        2           120.764619   
                                                               6           142.742602   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 0.0001        2           143.907106   
                                                               6           180.819376   
                                                 0.0010        2           142.842305   
                                                               6           179.499949   
                                                 0.0100        2           142.596723   
                                                               6           179.148274   

                                                                           Train MSE  \
Architecture                                     Learning Rate # of Heads              
L16/C32/C64/C128/C256/C256/C128/L64/L2           0.0001        2            0.000012   
                                                               6            0.000008   
                                                 0.0010        2            0.000004   
                                                               6         

In [14]:
# sorted
pd.options.display.float_format = '{:.2e}'.format
df.groupby(['Architecture', 'Learning Rate', '# of Heads']).mean().sort_values('Test MSE')[['Parameters', 'Train Time','Train MSE', 'Test MSE']]

Parameters  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-03      4              1371426   
                                                               8              2730238   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-03      4               581406   
                                                               12             1722062   
                                                               8              1151734   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      8              4833030   
                                                               4              2423590   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-03      12             4089050   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      12             7242470   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      4              1371426   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      8              1151734   
                                                               4               581406   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      8              2730238   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      8              1151734   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-04      8              2730238   
                                                               4              1371426   
                                                               12             4089050   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-04      12             7242470   
                                                               8              4833030   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      12             1722062   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-02      4              2423590   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      4               581406   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-04      4              2423590   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      12             1722062   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      12             4089050   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-02      8              4833030   
                                                               12             7242470   

                                                                           Train Time  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-03      4             1.30e+02   
                                                               8             1.54e+02   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-03      4             1.07e+02   
                                                               12            1.26e+02   
                                                               8             1.15e+02   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      8             1.99e+02   
                                                               4             1.59e+02   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-03      12            1.76e+02   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      12            2.36e+02   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      4             1.31e+02   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      8             1.15e+02   
                                                               4             1.04e+02   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      8             1.53e+02   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      8             1.19e+02   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-04      8     

In [9]:
df.groupby(['Architecture']).mean()

,Learning Rate,# of Heads,Trial,Parameters,Train Time,Train MSE,Test MSE
Architecture,,,,,,,
L16/C32/C64/C128/C256/C256/C128/L64/L2,0.0037,8,2,1151734,116.945098,0.000009,0.000014
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2,0.0037,8,2,2730238,153.482014,0.000008,0.000013
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2,0.0037,8,2,4833030,198.162713,0.000010,0.000016


In [10]:
df.groupby(['Learning Rate']).mean()

,# of Heads,Trial,Parameters,Train Time,Train MSE,Test MSE
Learning Rate,,,,,,
0.0001,8.0,2.0,2.905001e+06,157.767756,0.000006,0.000015
0.0010,8.0,2.0,2.905001e+06,155.822177,0.000006,0.000010
0.0100,8.0,2.0,2.905001e+06,154.999891,0.000014,0.000017


## 4. Study data transformations

In [12]:
saveDir = './results/modelTuning/transformTuning01/'

# build training plan
options = [True, False]
seedList = [1,2,3]
trainConfigs = []
for st in options:
    for lt in options:
        for s in seedList:
            trainConfigs.append({'Standard Scaler': st, 'Log Transform': lt, 'Trial': s})

# train models
for i,c in enumerate(trainConfigs):
    print(c)
    trainData, valData, testData = partitionGraphList(allGraphs, seed=c['Trial'])
    gcn = FeaStNet(architecture='L16/C32/C64/C128/C256/C512/C256/C128/L64/L2', heads=8, useConstraints=False)
    
    t = time()
    history = gcn.trainModel(trainData, valData, 
                             epochs=100,
                             lr=1e-3,
                             batchSize=256, 
                             flatten=True, 
                             logTrans=c['Log Transform'], 
                             ssTrans=c['Standard Scaler'], 
                             saveDir=saveDir+f'/{i:03}/')
    
    # test
    trainRes = gcn.testModel(trainData)
    c['Train MSE'] = trainRes['mse']
    
    testRes = gcn.testModel(testData)
    c['Test MSE'] = testRes['mse']

    # log
    df = pd.DataFrame(trainConfigs)
    df.to_csv(saveDir+'testResults.csv', index=False)

{'Standard Scaler': True, 'Log Transform': True, 'Trial': 1}
epoch: 0   trainLoss: 3.3730e+00   valLoss:3.2823e+00  time: 1.51e+00
epoch: 1   trainLoss: 2.8218e+00   valLoss:3.0822e+00  time: 1.39e+00
epoch: 2   trainLoss: 2.2056e+00   valLoss:3.1703e+00  time: 1.42e+00
epoch: 3   trainLoss: 1.7758e+00   valLoss:3.5928e+00  time: 1.42e+00
epoch: 4   trainLoss: 1.6130e+00   valLoss:4.2503e+00  time: 1.42e+00
epoch: 5   trainLoss: 1.3020e+00   valLoss:5.3417e+00  time: 1.42e+00
epoch: 6   trainLoss: 1.0758e+00   valLoss:4.0878e+00  time: 1.42e+00
epoch: 7   trainLoss: 8.7072e-01   valLoss:1.8933e+00  time: 1.41e+00
epoch: 8   trainLoss: 7.2386e-01   valLoss:1.3619e+00  time: 1.40e+00
epoch: 9   trainLoss: 6.6876e-01   valLoss:1.1203e+00  time: 1.40e+00
epoch: 10   trainLoss: 5.4587e-01   valLoss:1.4838e+00  time: 1.40e+00
epoch: 11   trainLoss: 5.4358e-01   valLoss:1.4048e+00  time: 1.40e+00
epoch: 12   trainLoss: 6.1722e-01   valLoss:1.3492e+00  time: 1.40e+00
epoch: 13   trainLoss: 5.4

epoch: 14   trainLoss: 5.5659e-01   valLoss:1.6764e+00  time: 1.36e+00
epoch: 15   trainLoss: 4.5251e-01   valLoss:1.1733e+00  time: 1.37e+00
epoch: 16   trainLoss: 4.9012e-01   valLoss:1.2659e+00  time: 1.37e+00
epoch: 17   trainLoss: 3.7525e-01   valLoss:1.1178e+00  time: 1.36e+00
epoch: 18   trainLoss: 3.6125e-01   valLoss:8.2096e-01  time: 1.37e+00
epoch: 19   trainLoss: 3.5300e-01   valLoss:8.3378e-01  time: 1.38e+00
epoch: 20   trainLoss: 3.0137e-01   valLoss:7.4509e-01  time: 1.39e+00
epoch: 21   trainLoss: 3.6224e-01   valLoss:1.0737e+00  time: 1.36e+00
epoch: 22   trainLoss: 4.2916e-01   valLoss:6.1926e-01  time: 1.36e+00
epoch: 23   trainLoss: 2.9637e-01   valLoss:1.3997e+00  time: 1.38e+00
epoch: 24   trainLoss: 2.5580e-01   valLoss:1.8752e+00  time: 1.36e+00
epoch: 25   trainLoss: 2.5444e-01   valLoss:1.0204e+00  time: 1.35e+00
epoch: 26   trainLoss: 2.5702e-01   valLoss:1.4832e+00  time: 1.36e+00
epoch: 27   trainLoss: 2.8049e-01   valLoss:5.8213e-01  time: 1.37e+00
epoch:

epoch: 29   trainLoss: 2.2512e-01   valLoss:1.1367e+00  time: 1.39e+00
epoch: 30   trainLoss: 2.0057e-01   valLoss:8.6688e-01  time: 1.39e+00
epoch: 31   trainLoss: 1.8110e-01   valLoss:9.4434e-01  time: 1.36e+00
epoch: 32   trainLoss: 1.7755e-01   valLoss:7.6961e-01  time: 1.37e+00
epoch: 33   trainLoss: 2.0327e-01   valLoss:8.9580e-01  time: 1.37e+00
epoch: 34   trainLoss: 2.0243e-01   valLoss:1.0368e+00  time: 1.39e+00
epoch: 35   trainLoss: 1.6783e-01   valLoss:8.1700e-01  time: 1.39e+00
epoch: 36   trainLoss: 1.7186e-01   valLoss:1.1758e+00  time: 1.37e+00
epoch: 37   trainLoss: 1.5880e-01   valLoss:1.3444e+00  time: 1.36e+00
epoch: 38   trainLoss: 2.1344e-01   valLoss:7.0263e-01  time: 1.40e+00
epoch: 39   trainLoss: 1.6603e-01   valLoss:6.7847e-01  time: 1.39e+00
epoch: 40   trainLoss: 1.6204e-01   valLoss:6.1914e-01  time: 1.39e+00
epoch: 41   trainLoss: 1.6656e-01   valLoss:9.6027e-01  time: 1.39e+00
epoch: 42   trainLoss: 1.6220e-01   valLoss:7.4473e-01  time: 1.37e+00
epoch:

epoch: 44   trainLoss: 6.5102e-02   valLoss:2.0946e-01  time: 1.38e+00
epoch: 45   trainLoss: 5.4802e-02   valLoss:1.8215e-01  time: 1.38e+00
epoch: 46   trainLoss: 6.5117e-02   valLoss:1.8247e-01  time: 1.39e+00
epoch: 47   trainLoss: 4.4463e-02   valLoss:1.5408e-01  time: 1.38e+00
epoch: 48   trainLoss: 4.5099e-02   valLoss:1.3581e-01  time: 1.40e+00
epoch: 49   trainLoss: 4.7042e-02   valLoss:1.7857e-01  time: 1.39e+00
epoch: 50   trainLoss: 3.9293e-02   valLoss:1.4398e-01  time: 1.39e+00
epoch: 51   trainLoss: 3.3178e-02   valLoss:1.4915e-01  time: 1.39e+00
epoch: 52   trainLoss: 3.5938e-02   valLoss:1.0956e-01  time: 1.40e+00
epoch: 53   trainLoss: 4.4962e-02   valLoss:1.3393e-01  time: 1.40e+00
epoch: 54   trainLoss: 3.5984e-02   valLoss:1.0602e-01  time: 1.39e+00
epoch: 55   trainLoss: 7.1875e-02   valLoss:2.7155e-01  time: 1.39e+00
epoch: 56   trainLoss: 5.3789e-02   valLoss:1.7352e-01  time: 1.39e+00
epoch: 57   trainLoss: 6.1844e-02   valLoss:2.0617e-01  time: 1.38e+00
epoch:

epoch: 59   trainLoss: 5.6927e-02   valLoss:1.2672e-01  time: 1.40e+00
epoch: 60   trainLoss: 6.5265e-02   valLoss:2.6609e-01  time: 1.42e+00
epoch: 61   trainLoss: 6.2595e-02   valLoss:1.7295e-01  time: 1.38e+00
epoch: 62   trainLoss: 4.0756e-02   valLoss:1.3160e-01  time: 1.36e+00
epoch: 63   trainLoss: 5.4328e-02   valLoss:1.1451e-01  time: 1.37e+00
epoch: 64   trainLoss: 4.8023e-02   valLoss:2.2792e-01  time: 1.37e+00
epoch: 65   trainLoss: 4.2396e-02   valLoss:1.7269e-01  time: 1.38e+00
epoch: 66   trainLoss: 5.0499e-02   valLoss:1.4735e-01  time: 1.40e+00
epoch: 67   trainLoss: 4.6145e-02   valLoss:1.5920e-01  time: 1.39e+00
epoch: 68   trainLoss: 5.0126e-02   valLoss:1.3352e-01  time: 1.36e+00
epoch: 69   trainLoss: 4.6374e-02   valLoss:1.6249e-01  time: 1.36e+00
epoch: 70   trainLoss: 3.7793e-02   valLoss:1.4954e-01  time: 1.36e+00
epoch: 71   trainLoss: 4.2795e-02   valLoss:1.1312e-01  time: 1.35e+00
epoch: 72   trainLoss: 3.7242e-02   valLoss:1.5316e-01  time: 1.39e+00
epoch:

...

epoch: 89   trainLoss: 1.9575e-03   valLoss:4.0118e-03  time: 1.37e+00
epoch: 90   trainLoss: 2.0399e-03   valLoss:3.1952e-03  time: 1.38e+00
epoch: 91   trainLoss: 1.8342e-03   valLoss:3.6334e-03  time: 1.36e+00
epoch: 92   trainLoss: 2.0697e-03   valLoss:3.8438e-03  time: 1.37e+00
epoch: 93   trainLoss: 1.9407e-03   valLoss:7.3136e-03  time: 1.34e+00
epoch: 94   trainLoss: 2.2814e-03   valLoss:4.2729e-03  time: 1.37e+00
epoch: 95   trainLoss: 2.0702e-03   valLoss:3.5332e-03  time: 1.37e+00
epoch: 96   trainLoss: 2.1712e-03   valLoss:5.9708e-03  time: 1.38e+00
epoch: 97   trainLoss: 2.6865e-03   valLoss:6.9248e-03  time: 1.36e+00
epoch: 98   trainLoss: 2.0765e-03   valLoss:3.3537e-03  time: 1.35e+00
epoch: 99   trainLoss: 1.9917e-03   valLoss:3.5988e-03  time: 1.35e+00
loading checkpoint 83
{'Standard Scaler': False, 'Log Transform': True, 'Trial': 2}
epoch: 0   trainLoss: 4.3075e-02   valLoss:1.5705e-02  time: 1.45e+00
epoch: 1   trainLoss: 1.8261e-02   valLoss:1.4012e-02  time: 1.34

epoch: 3   trainLoss: 1.1113e-02   valLoss:1.4486e-02  time: 1.35e+00
epoch: 4   trainLoss: 8.8586e-03   valLoss:1.7454e-02  time: 1.36e+00
epoch: 5   trainLoss: 8.2867e-03   valLoss:1.6958e-02  time: 1.35e+00
epoch: 6   trainLoss: 7.9375e-03   valLoss:1.5161e-02  time: 1.36e+00
epoch: 7   trainLoss: 7.4290e-03   valLoss:1.2054e-02  time: 1.39e+00
epoch: 8   trainLoss: 6.5457e-03   valLoss:8.3568e-03  time: 1.34e+00
epoch: 9   trainLoss: 6.1457e-03   valLoss:7.6493e-03  time: 1.34e+00
epoch: 10   trainLoss: 6.2314e-03   valLoss:8.3969e-03  time: 1.36e+00
epoch: 11   trainLoss: 5.7527e-03   valLoss:7.7903e-03  time: 1.35e+00
epoch: 12   trainLoss: 5.1795e-03   valLoss:7.0574e-03  time: 1.36e+00
epoch: 13   trainLoss: 4.9764e-03   valLoss:7.6690e-03  time: 1.37e+00
epoch: 14   trainLoss: 4.4657e-03   valLoss:6.8962e-03  time: 1.38e+00
epoch: 15   trainLoss: 4.3818e-03   valLoss:7.8073e-03  time: 1.37e+00
epoch: 16   trainLoss: 4.3742e-03   valLoss:7.9013e-03  time: 1.37e+00
epoch: 17   t

epoch: 18   trainLoss: 2.9578e-04   valLoss:3.1324e-04  time: 1.38e+00
epoch: 19   trainLoss: 2.6317e-04   valLoss:4.1665e-04  time: 1.37e+00
epoch: 20   trainLoss: 3.0718e-04   valLoss:3.3147e-04  time: 1.37e+00
epoch: 21   trainLoss: 2.7266e-04   valLoss:4.0883e-04  time: 1.36e+00
epoch: 22   trainLoss: 2.9220e-04   valLoss:3.2682e-04  time: 1.38e+00
epoch: 23   trainLoss: 3.2008e-04   valLoss:4.0214e-04  time: 1.36e+00
epoch: 24   trainLoss: 2.9862e-04   valLoss:3.1881e-04  time: 1.37e+00
epoch: 25   trainLoss: 3.0974e-04   valLoss:3.1123e-04  time: 1.39e+00
epoch: 26   trainLoss: 2.9072e-04   valLoss:2.5207e-04  time: 1.39e+00
epoch: 27   trainLoss: 2.8079e-04   valLoss:2.6844e-04  time: 1.38e+00
epoch: 28   trainLoss: 2.3821e-04   valLoss:2.7355e-04  time: 1.38e+00
epoch: 29   trainLoss: 2.1119e-04   valLoss:2.4250e-04  time: 1.39e+00
epoch: 30   trainLoss: 1.9689e-04   valLoss:2.2081e-04  time: 1.37e+00
epoch: 31   trainLoss: 1.7530e-04   valLoss:1.9640e-04  time: 1.38e+00
epoch:

epoch: 33   trainLoss: 1.6143e-04   valLoss:2.0248e-04  time: 1.36e+00
epoch: 34   trainLoss: 1.8597e-04   valLoss:2.3329e-04  time: 1.39e+00
epoch: 35   trainLoss: 1.4909e-04   valLoss:2.3027e-04  time: 1.37e+00
epoch: 36   trainLoss: 1.9278e-04   valLoss:3.1618e-04  time: 1.38e+00
epoch: 37   trainLoss: 2.0603e-04   valLoss:2.4822e-04  time: 1.38e+00
epoch: 38   trainLoss: 1.9519e-04   valLoss:3.0668e-04  time: 1.38e+00
epoch: 39   trainLoss: 1.9434e-04   valLoss:2.1384e-04  time: 1.38e+00
epoch: 40   trainLoss: 1.6047e-04   valLoss:2.3379e-04  time: 1.37e+00
epoch: 41   trainLoss: 1.7518e-04   valLoss:1.8848e-04  time: 1.38e+00
epoch: 42   trainLoss: 1.7148e-04   valLoss:2.1829e-04  time: 1.38e+00
epoch: 43   trainLoss: 1.5766e-04   valLoss:1.9207e-04  time: 1.36e+00
epoch: 44   trainLoss: 1.7516e-04   valLoss:2.2328e-04  time: 1.38e+00
epoch: 45   trainLoss: 1.8056e-04   valLoss:2.3076e-04  time: 1.37e+00
epoch: 46   trainLoss: 1.7187e-04   valLoss:2.0350e-04  time: 1.38e+00
epoch:

epoch: 48   trainLoss: 1.7613e-04   valLoss:2.5506e-04  time: 1.38e+00
epoch: 49   trainLoss: 2.0897e-04   valLoss:1.6411e-04  time: 1.37e+00
epoch: 50   trainLoss: 1.5314e-04   valLoss:1.3917e-04  time: 1.38e+00
epoch: 51   trainLoss: 1.5180e-04   valLoss:1.5048e-04  time: 1.38e+00
epoch: 52   trainLoss: 1.2617e-04   valLoss:1.1885e-04  time: 1.37e+00
epoch: 53   trainLoss: 1.2125e-04   valLoss:1.1427e-04  time: 1.37e+00
epoch: 54   trainLoss: 1.1028e-04   valLoss:1.5378e-04  time: 1.37e+00
epoch: 55   trainLoss: 1.3070e-04   valLoss:1.0152e-04  time: 1.38e+00
epoch: 56   trainLoss: 1.2761e-04   valLoss:1.1672e-04  time: 1.37e+00
epoch: 57   trainLoss: 1.1555e-04   valLoss:1.0928e-04  time: 1.39e+00
epoch: 58   trainLoss: 1.1878e-04   valLoss:1.1375e-04  time: 1.38e+00
epoch: 59   trainLoss: 1.1859e-04   valLoss:1.0365e-04  time: 1.38e+00
epoch: 60   trainLoss: 1.1910e-04   valLoss:1.0609e-04  time: 1.38e+00
epoch: 61   trainLoss: 1.1333e-04   valLoss:1.1802e-04  time: 1.37e+00
epoch:

In [13]:
df.groupby(['Standard Scaler', 'Log Transform']).mean()[['Train MSE', 'Test MSE']]

Train MSE  Test MSE
Standard Scaler Log Transform                     
False           False           0.000101  0.000118
                True            0.000045  0.000058
True            False           0.000011  0.000020
                True            0.000015  0.000025

## 5. Should binary loads and BCs be used as features?

In [7]:
saveDir = './results/modelTuning/constraintStudy02/'

# build training plan
options = [True, False]
seedList = [1,2,3]
trainConfigs = []
for uc in options:
    for ul in options:
        for s in seedList:
            trainConfigs.append({'Include supports': uc, 'Include loads': ul, 'Trial': s})

# train models
for i,c in enumerate(trainConfigs):
    print(c)
    
    supportDims = [0,2] if c['Include supports'] else []
    loadDims = [2] if c['Include loads'] else []
    useXFeatures = c['Include supports'] or c['Include loads']
    
    allGraphsUnfiltered = loadConmechGraphs(dataDir, supportDims=supportDims, loadDims=loadDims)
    allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
    trainData, valData, testData = partitionGraphList(allGraphs, seed=c['Trial'])
    gcn = FeaStNet(architecture='L16/C32/C64/C128/C256/C512/C256/C128/L64/L2', heads=8, useXFeatures=useXFeatures,
                  numXFeatures = len(supportDims)+len(loadDims))
    
    t = time()
    history = gcn.trainModel(trainData, valData, 
                             epochs=100,
                             lr=1e-3,
                             batchSize=256, 
                             flatten=True, 
                             logTrans=False, 
                             ssTrans=True,
                             binaryLoads=True,
                             saveDir=saveDir+f'/{i:03}/')
    
    # test
    trainRes = gcn.testModel(trainData)
    c['Train MSE'] = trainRes['mse']
    
    testRes = gcn.testModel(testData)
    c['Test MSE'] = testRes['mse']
    print(c['Test MSE'])

    # log
    df = pd.DataFrame(trainConfigs)
    df.to_csv(saveDir+'testResults.csv', index=False)

{'Include supports': True, 'Include loads': True, 'Trial': 1}
epoch: 0   trainLoss: 9.0584e-01   valLoss:8.9152e-01  time: 1.68e+00
epoch: 1   trainLoss: 7.5908e-01   valLoss:8.7954e-01  time: 1.54e+00
epoch: 2   trainLoss: 6.7754e-01   valLoss:8.9765e-01  time: 1.54e+00
epoch: 3   trainLoss: 5.6128e-01   valLoss:9.4459e-01  time: 1.54e+00
epoch: 4   trainLoss: 4.5913e-01   valLoss:1.1560e+00  time: 1.53e+00
epoch: 5   trainLoss: 4.3044e-01   valLoss:1.5686e+00  time: 1.56e+00
epoch: 6   trainLoss: 3.8242e-01   valLoss:2.0498e+00  time: 1.58e+00
epoch: 7   trainLoss: 3.3445e-01   valLoss:3.3245e+00  time: 1.54e+00
epoch: 8   trainLoss: 3.1042e-01   valLoss:2.8734e+00  time: 1.59e+00
epoch: 9   trainLoss: 2.8672e-01   valLoss:2.2782e+00  time: 1.58e+00
epoch: 10   trainLoss: 2.8840e-01   valLoss:2.5652e+00  time: 1.54e+00
epoch: 11   trainLoss: 2.2239e-01   valLoss:1.9741e+00  time: 1.52e+00
epoch: 12   trainLoss: 1.9934e-01   valLoss:3.8419e+00  time: 1.56e+00
epoch: 13   trainLoss: 1.

epoch: 14   trainLoss: 2.6717e-01   valLoss:4.9152e-01  time: 1.59e+00
epoch: 15   trainLoss: 2.6069e-01   valLoss:4.1711e-01  time: 1.54e+00
epoch: 16   trainLoss: 2.6839e-01   valLoss:3.9196e-01  time: 1.54e+00
epoch: 17   trainLoss: 2.1547e-01   valLoss:3.6831e-01  time: 1.58e+00
epoch: 18   trainLoss: 2.2330e-01   valLoss:3.3143e-01  time: 1.56e+00
epoch: 19   trainLoss: 1.9617e-01   valLoss:4.1378e-01  time: 1.57e+00
epoch: 20   trainLoss: 1.8097e-01   valLoss:4.4947e-01  time: 1.54e+00
epoch: 21   trainLoss: 1.5496e-01   valLoss:3.9697e-01  time: 1.60e+00
epoch: 22   trainLoss: 1.4755e-01   valLoss:4.1848e-01  time: 1.58e+00
epoch: 23   trainLoss: 1.5005e-01   valLoss:3.2318e-01  time: 1.55e+00
epoch: 24   trainLoss: 1.3848e-01   valLoss:2.2741e-01  time: 1.58e+00
epoch: 25   trainLoss: 1.6367e-01   valLoss:7.0711e-01  time: 1.60e+00
epoch: 26   trainLoss: 1.3852e-01   valLoss:5.3873e-01  time: 1.59e+00
epoch: 27   trainLoss: 1.3780e-01   valLoss:9.3568e-01  time: 1.58e+00
epoch:

epoch: 29   trainLoss: 8.4257e-02   valLoss:3.0855e-01  time: 1.54e+00
epoch: 30   trainLoss: 7.8671e-02   valLoss:4.3367e+00  time: 1.55e+00
epoch: 31   trainLoss: 7.5334e-02   valLoss:1.7241e+00  time: 1.59e+00
epoch: 32   trainLoss: 9.7585e-02   valLoss:3.4411e+00  time: 1.57e+00
epoch: 33   trainLoss: 8.6749e-02   valLoss:1.2309e+00  time: 1.54e+00
epoch: 34   trainLoss: 8.6901e-02   valLoss:3.9905e+00  time: 1.53e+00
epoch: 35   trainLoss: 8.3450e-02   valLoss:6.7355e-01  time: 1.56e+00
epoch: 36   trainLoss: 7.7529e-02   valLoss:9.9912e-01  time: 1.52e+00
epoch: 37   trainLoss: 6.2280e-02   valLoss:1.7199e+00  time: 1.57e+00
epoch: 38   trainLoss: 6.3147e-02   valLoss:1.8809e+00  time: 1.57e+00
epoch: 39   trainLoss: 5.7682e-02   valLoss:2.5309e+00  time: 1.58e+00
epoch: 40   trainLoss: 6.0704e-02   valLoss:4.5934e-01  time: 1.54e+00
epoch: 41   trainLoss: 6.1323e-02   valLoss:3.6928e+00  time: 1.58e+00
epoch: 42   trainLoss: 6.2132e-02   valLoss:3.9872e-01  time: 1.55e+00
epoch:

epoch: 44   trainLoss: 7.4760e-02   valLoss:3.2440e-01  time: 1.58e+00
epoch: 45   trainLoss: 7.0559e-02   valLoss:3.2179e-01  time: 1.58e+00
epoch: 46   trainLoss: 6.7814e-02   valLoss:2.6118e-01  time: 1.60e+00
epoch: 47   trainLoss: 6.4744e-02   valLoss:2.5728e-01  time: 1.58e+00
epoch: 48   trainLoss: 7.6611e-02   valLoss:2.0547e-01  time: 1.59e+00
epoch: 49   trainLoss: 5.5658e-02   valLoss:1.7934e-01  time: 1.60e+00
epoch: 50   trainLoss: 5.7086e-02   valLoss:1.8601e-01  time: 1.56e+00
epoch: 51   trainLoss: 5.3663e-02   valLoss:1.7125e-01  time: 1.56e+00
epoch: 52   trainLoss: 4.3265e-02   valLoss:1.4178e-01  time: 1.59e+00
epoch: 53   trainLoss: 5.4172e-02   valLoss:1.9837e-01  time: 1.60e+00
epoch: 54   trainLoss: 4.4548e-02   valLoss:2.0789e-01  time: 1.59e+00
epoch: 55   trainLoss: 5.5641e-02   valLoss:3.0482e-01  time: 1.56e+00
epoch: 56   trainLoss: 4.0326e-02   valLoss:2.2953e-01  time: 1.59e+00
epoch: 57   trainLoss: 4.5448e-02   valLoss:2.4611e-01  time: 1.61e+00
epoch:

epoch: 59   trainLoss: 4.7440e-02   valLoss:2.3042e-01  time: 1.56e+00
epoch: 60   trainLoss: 4.8730e-02   valLoss:2.3527e-01  time: 1.57e+00
epoch: 61   trainLoss: 5.7658e-02   valLoss:1.6288e-01  time: 1.55e+00
epoch: 62   trainLoss: 5.5563e-02   valLoss:4.8555e-01  time: 1.60e+00
epoch: 63   trainLoss: 4.5538e-02   valLoss:1.9375e-01  time: 1.59e+00
epoch: 64   trainLoss: 5.6212e-02   valLoss:2.3301e-01  time: 1.57e+00
epoch: 65   trainLoss: 4.5871e-02   valLoss:3.2648e-01  time: 1.57e+00
epoch: 66   trainLoss: 5.0374e-02   valLoss:2.2465e-01  time: 1.59e+00
epoch: 67   trainLoss: 4.1814e-02   valLoss:2.7003e-01  time: 1.60e+00
epoch: 68   trainLoss: 3.7841e-02   valLoss:1.4377e-01  time: 1.70e+00
epoch: 69   trainLoss: 4.2796e-02   valLoss:1.9843e-01  time: 1.76e+00
epoch: 70   trainLoss: 3.3320e-02   valLoss:1.3828e-01  time: 1.58e+00
epoch: 71   trainLoss: 3.3814e-02   valLoss:1.5931e-01  time: 1.62e+00
epoch: 72   trainLoss: 3.2435e-02   valLoss:1.7701e-01  time: 1.75e+00
epoch:

...

epoch: 89   trainLoss: 5.0767e-02   valLoss:2.2939e-01  time: 1.59e+00
epoch: 90   trainLoss: 4.0574e-02   valLoss:3.7473e-01  time: 1.60e+00
epoch: 91   trainLoss: 4.2985e-02   valLoss:4.2723e-01  time: 1.60e+00
epoch: 92   trainLoss: 4.7107e-02   valLoss:2.6922e-01  time: 1.57e+00
epoch: 93   trainLoss: 4.5084e-02   valLoss:3.2052e-01  time: 1.58e+00
epoch: 94   trainLoss: 5.9395e-02   valLoss:5.0380e-01  time: 1.57e+00
epoch: 95   trainLoss: 3.9353e-02   valLoss:2.0294e-01  time: 1.55e+00
epoch: 96   trainLoss: 4.5140e-02   valLoss:3.2779e-01  time: 1.56e+00
epoch: 97   trainLoss: 3.4447e-02   valLoss:2.8185e-01  time: 1.56e+00
epoch: 98   trainLoss: 4.9976e-02   valLoss:2.3871e-01  time: 1.56e+00
epoch: 99   trainLoss: 4.7606e-02   valLoss:3.0049e-01  time: 1.57e+00
loading checkpoint 76
1.4946521e-05
{'Include supports': False, 'Include loads': True, 'Trial': 2}
epoch: 0   trainLoss: 9.4855e-01   valLoss:8.5025e-01  time: 1.64e+00
epoch: 1   trainLoss: 7.7141e-01   valLoss:8.2270e

epoch: 2   trainLoss: 6.1041e-01   valLoss:8.7822e-01  time: 1.54e+00
epoch: 3   trainLoss: 5.1640e-01   valLoss:1.0276e+00  time: 1.54e+00
epoch: 4   trainLoss: 4.7839e-01   valLoss:1.3889e+00  time: 1.53e+00
epoch: 5   trainLoss: 4.4716e-01   valLoss:1.2094e+00  time: 1.57e+00
epoch: 6   trainLoss: 4.3062e-01   valLoss:1.0356e+00  time: 1.58e+00
epoch: 7   trainLoss: 3.6897e-01   valLoss:7.2008e-01  time: 1.56e+00
epoch: 8   trainLoss: 3.1675e-01   valLoss:5.3479e-01  time: 1.56e+00
epoch: 9   trainLoss: 2.8630e-01   valLoss:5.6271e-01  time: 1.57e+00
epoch: 10   trainLoss: 2.5139e-01   valLoss:6.2486e-01  time: 1.57e+00
epoch: 11   trainLoss: 2.1506e-01   valLoss:5.5077e-01  time: 1.57e+00
epoch: 12   trainLoss: 1.8499e-01   valLoss:6.8608e-01  time: 1.56e+00
epoch: 13   trainLoss: 1.8071e-01   valLoss:5.6875e-01  time: 1.57e+00
epoch: 14   trainLoss: 1.6807e-01   valLoss:4.3430e-01  time: 1.61e+00
epoch: 15   trainLoss: 1.5457e-01   valLoss:6.5846e-01  time: 1.58e+00
epoch: 16   tr

epoch: 17   trainLoss: 1.6746e-01   valLoss:8.8074e-01  time: 1.54e+00
epoch: 18   trainLoss: 1.3089e-01   valLoss:3.6583e-01  time: 1.55e+00
epoch: 19   trainLoss: 1.1894e-01   valLoss:7.3294e-01  time: 1.55e+00
epoch: 20   trainLoss: 1.0137e-01   valLoss:1.0473e+00  time: 1.55e+00
epoch: 21   trainLoss: 1.0230e-01   valLoss:8.1095e-01  time: 1.56e+00
epoch: 22   trainLoss: 1.1686e-01   valLoss:4.6578e-01  time: 1.56e+00
epoch: 23   trainLoss: 1.2266e-01   valLoss:1.2597e+00  time: 1.53e+00
epoch: 24   trainLoss: 1.1906e-01   valLoss:5.7399e-01  time: 1.56e+00
epoch: 25   trainLoss: 1.1172e-01   valLoss:3.5239e-01  time: 1.56e+00
epoch: 26   trainLoss: 1.0220e-01   valLoss:1.0835e+00  time: 1.58e+00
epoch: 27   trainLoss: 9.1641e-02   valLoss:1.0958e+00  time: 1.56e+00
epoch: 28   trainLoss: 8.6803e-02   valLoss:4.8751e-01  time: 1.57e+00
epoch: 29   trainLoss: 7.6470e-02   valLoss:3.2909e-01  time: 1.54e+00
epoch: 30   trainLoss: 7.7341e-02   valLoss:3.0361e-01  time: 1.58e+00
epoch:

epoch: 32   trainLoss: 1.2724e-01   valLoss:1.2235e+00  time: 1.67e+00
epoch: 33   trainLoss: 1.0441e-01   valLoss:8.6374e-01  time: 1.72e+00
epoch: 34   trainLoss: 9.6027e-02   valLoss:8.0731e-01  time: 1.69e+00
epoch: 35   trainLoss: 9.3694e-02   valLoss:4.4026e-01  time: 1.57e+00
epoch: 36   trainLoss: 1.0396e-01   valLoss:4.1762e-01  time: 1.66e+00
epoch: 37   trainLoss: 9.0314e-02   valLoss:5.9576e-01  time: 1.79e+00
epoch: 38   trainLoss: 9.4802e-02   valLoss:4.1437e-01  time: 1.70e+00
epoch: 39   trainLoss: 7.0084e-02   valLoss:3.6726e-01  time: 1.60e+00
epoch: 40   trainLoss: 7.2228e-02   valLoss:2.8003e-01  time: 1.73e+00
epoch: 41   trainLoss: 5.8095e-02   valLoss:4.0118e-01  time: 1.76e+00
epoch: 42   trainLoss: 5.8932e-02   valLoss:3.3899e-01  time: 1.62e+00
epoch: 43   trainLoss: 6.9146e-02   valLoss:4.2625e-01  time: 1.59e+00
epoch: 44   trainLoss: 7.6323e-02   valLoss:8.1534e-01  time: 1.71e+00
epoch: 45   trainLoss: 8.3838e-02   valLoss:4.0271e-01  time: 1.71e+00
epoch:

epoch: 47   trainLoss: 7.5598e-02   valLoss:1.8984e-01  time: 1.57e+00
epoch: 48   trainLoss: 8.2037e-02   valLoss:2.2164e-01  time: 1.56e+00
epoch: 49   trainLoss: 6.4042e-02   valLoss:1.7390e-01  time: 1.56e+00
epoch: 50   trainLoss: 5.8686e-02   valLoss:1.5222e-01  time: 1.53e+00
epoch: 51   trainLoss: 6.3016e-02   valLoss:1.5794e-01  time: 1.58e+00
epoch: 52   trainLoss: 5.1646e-02   valLoss:1.4622e-01  time: 1.55e+00
epoch: 53   trainLoss: 4.6454e-02   valLoss:1.3692e-01  time: 1.56e+00
epoch: 54   trainLoss: 5.3435e-02   valLoss:1.2610e-01  time: 1.57e+00
epoch: 55   trainLoss: 5.2976e-02   valLoss:1.5739e-01  time: 1.56e+00
epoch: 56   trainLoss: 4.7387e-02   valLoss:1.5123e-01  time: 1.56e+00
epoch: 57   trainLoss: 4.9384e-02   valLoss:1.0778e-01  time: 1.54e+00
epoch: 58   trainLoss: 8.0478e-02   valLoss:1.0822e-01  time: 1.54e+00
epoch: 59   trainLoss: 4.6033e-02   valLoss:1.1267e-01  time: 1.53e+00
epoch: 60   trainLoss: 5.0489e-02   valLoss:1.3578e-01  time: 1.57e+00
epoch:

In [10]:
df.groupby(['Include supports', 'Include loads']).mean()[['Train MSE', 'Test MSE']]

Train MSE  Test MSE
Include supports Include loads                     
False            False           0.000005  0.000012
                 True            0.000006  0.000012
True             False           0.000007  0.000011
                 True            0.000005  0.000010

In [11]:
df

,Include supports,Include loads,Trial,Train MSE,Test MSE
0,True,True,1,0.000006,0.000012
1,True,True,2,0.000005,0.000010
2,True,True,3,0.000004,0.000008
3,True,False,1,0.000005,0.000010
4,True,False,2,0.000009,0.000014
5,True,False,3,0.000006,0.000009
6,False,True,1,0.000009,0.000015
7,False,True,2,0.000003,0.000009
8,False,True,3,0.000006,0.000012
9,False,False,1,0.000005,0.000012


## Load a few data files

In [9]:
pd.options.display.float_format = '{:.2e}'.format
df2 = pd.read_csv('./results/modelTuning/architectureTuning02/testResults.csv')
df3 = pd.read_csv('./results/modelTuning/architectureTuning03/testResults.csv')
df = pd.concat([df2.reset_index(drop=True), df3.reset_index(drop=True)], axis= 0)
df.groupby(['Architecture', 'Learning Rate', '# of Heads']).mean()[['Parameters', 'Train Time','Train MSE', 'Test MSE']]

Parameters  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      2               296242   
                                                               4               581406   
                                                               6               866570   
                                                               8              1151734   
                                                               12             1722062   
                                                 1.00e-03      2               296242   
                                                               4               581406   
                                                               6               866570   
                                                               8              1151734   
                                                               12             1722062   
                                                 1.00e-02      2               296242   
                                                               4               581406   
                                                               6               866570   
                                                               8              1151734   
                                                               12             1722062   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-04      2               692020   
                                                               4              1371426   
                                                               6              2050832   
                                                               8              2730238   
                                                               12             4089050   
                                                 1.00e-03      2               692020   
                                                               4              1371426   
                                                               6              2050832   
                                                               8              2730238   
                                                               12             4089050   
                                                 1.00e-02      2               692020   
                                                               4              1371426   
                                                               6              2050832   
                                                               8              2730238   
                                                               12             4089050   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-04      2              1218870   
                                                               4              2423590   
                                                               6              3628310   
                                                               8              4833030   
                                                               12             7242470   
                                                 1.00e-03      2              1218870   
                                                               4              2423590   
                                                               6              3628310   
                                                               8              4833030   
                                                               12             7242470   
                                                 1.00e-02      2              1218870   
                                                               4              2423590   
                                                               6              3628310   
                                                               8      

In [10]:
df.groupby(['Architecture', 'Learning Rate', '# of Heads']).mean().sort_values('Test MSE')[['Parameters', 'Train Time','Train MSE', 'Test MSE']]

Parameters  \
Architecture                                     Learning Rate # of Heads               
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      6              3628310   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-03      4              1371426   
                                                               8              2730238   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-03      2               296242   
                                                               4               581406   
                                                               12             1722062   
                                                               8              1151734   
                                                               6               866570   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      8              4833030   
                                                               2              1218870   
                                                               4              2423590   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-03      12             4089050   
                                                               2               692020   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-03      12             7242470   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      2               296242   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      4              1371426   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      8              1151734   
                                                               4               581406   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      8              2730238   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-02      2              1218870   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      8              1151734   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-04      8              2730238   
                                                               4              1371426   
                                                               12             4089050   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-04      12             7242470   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      6               866570   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-04      2               692020   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-04      8              4833030   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      6               866570   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-04      6              2050832   
                                                 1.00e-02      2               692020   
                                                 1.00e-03      6              2050832   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      12             1722062   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-02      4              2423590   
                                                 1.00e-04      6              3628310   
                                                               2              1218870   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      6              2050832   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-02      6              3628310   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-04      4               581406   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-04      4              2423590   
L16/C32/C64/C128/C256/C256/C128/L64/L2           1.00e-02      12             1722062   
                                                 1.00e-04      2               296242   
L16/C32/C64/C128/C256/C512/C256/C128/L64/L2      1.00e-02      12             4089050   
L16/C32/C64/C128/C256/C512/C512/C256/C128/L64/L2 1.00e-02      8      